## 1. 사용할 패키지 불러오기

In [1]:
import pandas as pd
from glob import glob
import numpy as np
import re

## 2. 데이터 불러오기

In [27]:
xlsx_list = glob("./data/table_data/*.xlsx")
data = pd.read_excel(xlsx_list[0])
for xlsx_file in xlsx_list[1:]:
    data = pd.concat([data, pd.read_excel(xlsx_file)], axis=0).reset_index(drop = True)

data = data.iloc[:, 1:10]
data = data.loc[-data['이미지 고유 번호'].isna(), :].reset_index(drop = True)
data.head()

,작가명,제목,재료,사이즈,제작년도,판매가격,판매일자와판매처,이미지 고유 번호,작품 판매 횟수
0,임상진 Lim SangChin (1935~2013),무제,1983 캔버스에 아크릴,50☓60.6cm,1983.0,600000.0,2020-11-11 에이옥션,380410.0,1.0
1,정술원 Jung SulWon (1885~1959),화조,비단에 수묵담채,25☓24.5cm,NaN,400000.0,2020-11-11 에이옥션,380460.0,1.0
2,정주상 Jeong JuSang (1925~2012),심정흥장 (선면),종이에 먹,56☓17.5cm,NaN,100000.0,2020-11-11 에이옥션,380491.0,1.0
3,이양원 Lee YangWon (1944~),풍속도,종이에 수묵담채,55☓23.5cm,NaN,360000.0,2020-11-11 에이옥션,380417.0,1.0
4,이외수 Lee OiSoo (1946~),사람과 사람들,종이에 채색,25.5☓18.5cm,NaN,240000.0,2020-11-11 에이옥션,380391.0,1.0


## 3. 데이터 전처리

### (1) 작품 판매 횟수가 공백인 경우 1로 처리

In [28]:
data['작품 판매 횟수'][data['작품 판매 횟수'].isna()] = 1

/tmp/ipykernel_15432/3825792433.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['작품 판매 횟수'][data['작품 판매 횟수'].isna()] = 1


### (2) 제작년도의 경우 절반 이상이 공백으로 제거

In [29]:
del data['제작년도']

### (3) 작가 미상 여부 확인 변수 추가

In [30]:
data['작가미상여부'] = 0
data['작가미상여부'][data['작가명'] == '작가미상 Anonymous'] = 1

/tmp/ipykernel_15432/3541825850.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['작가미상여부'][data['작가명'] == '작가미상 Anonymous'] = 1


### (4) 작가 생존 여부 확인 변수 추가

In [31]:
data['작가생존여부'] = '생존'
data['작가생존여부'][data['작가명'] == '작가미상 Anonymous'] = 1

for i in range(len(data['작가생존여부'])):
    painter = data['작가명'][i]
    if painter[-1] != ')':
        data['작가생존여부'][i] = '알수없음'
    elif len(painter.split('(')[-1].split('~')[-1]) > 3:
        data['작가생존여부'][i] = '사망'
        

/tmp/ipykernel_15432/1162102882.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['작가생존여부'][data['작가명'] == '작가미상 Anonymous'] = 1
/tmp/ipykernel_15432/1162102882.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['작가생존여부'][i] = '사망'
/tmp/ipykernel_15432/1162102882.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['작가생존여부'][i] = '알수없음'


### (5) 재료에서 숫자 및 띄어쓰기 제거

##

In [32]:
data['재료'][data['재료'].isna()] = '알수없음'

def ingredient_preprocess(ingredient):
    new_str = re.sub(r"[0-9]", "", str(ingredient))
    new_str = new_str.replace(' ', '')
    if len(new_str) < 1:
        new_str = '알수없음'
    return new_str

# data['재료'] = data['재료'].map(ingredient_preprocess)
for i in range(len(data['재료'])):
    data['재료'][i] = ingredient_preprocess(data['재료'][i])

/tmp/ipykernel_15432/2469359434.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['재료'][data['재료'].isna()] = '알수없음'
/tmp/ipykernel_15432/2469359434.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['재료'][i] = ingredient_preprocess(data['재료'][i])


In [33]:
data['재료'][data['재료'] == 'oiloncanvas'] = '캔버스에유채'
data['재료'][data['재료'] == 'acryliconcanvas'] = '캔버스에아크릴'
data['재료'][data['재료'] == 'inkandcoloronpaper'] = '종에이수묵담채'
data['재료'][data['재료'] == '종이에수묵담채InkandColoronPaper'] = '종에이수묵담채'
data['재료'][data['재료'] == 'inkonpaper'] = '종이에유채'


/tmp/ipykernel_15432/4004730277.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['재료'][data['재료'] == 'oiloncanvas'] = '캔버스에유채'
/tmp/ipykernel_15432/4004730277.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['재료'][data['재료'] == 'acryliconcanvas'] = '캔버스에아크릴'
/tmp/ipykernel_15432/4004730277.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['재료'][data['재료'] == 'inkandcoloronpaper'] = '종에이수묵담채'
/tmp/ipykernel_15432/

In [34]:
from collections import Counter
most_things = Counter(data['재료']).most_common(20)
most_things = [thing for (thing, count) in most_things]

def thing_change(thing):
    if thing not in most_things:
        return '기타'
    else:
        return thing

data['재료'] = data['재료'].map(thing_change)

### (6) 사이즈 가로, 세로 추출

In [35]:
def extract_width(size):
    try:
        if len(re.sub(r"[^0-9, .]", "", size)) < 3:
            return 0
        else:
            if '×' in size:
                width = size.split('×')[0]
                width = re.sub(r"[^0-9, .]", "", width)
            else:
                width = size.split('☓')[0]
                width = re.sub(r"[^0-9, .]", "", width)
        return float(width)
    except:
        return 0

def extract_height(size):
    try:
        if len(re.sub(r"[^0-9, .]", "", size)) < 3:
            return 0
        else:
            if '×' in size:
                height = size.split('×')[1]
                height = re.sub(r"[^0-9, .]", "", height)
            else:
                height = size.split('☓')[1]
                height = re.sub(r"[^0-9, .]", "", height)
        return float(height)
    except:
        return 0

data['가로'] = 0
data['세로'] = 0
data['사이즈'][data['사이즈'].isna()] = '알수없음'

for i in range(len(data['사이즈'])):
    if len(data['사이즈'][i]) > 2 and len(data['사이즈'][i]) < 15:
        data['가로'][i] = extract_height(data['사이즈'][i])
        data['세로'][i] = extract_height(data['사이즈'][i])

/tmp/ipykernel_15432/905040304.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['사이즈'][data['사이즈'].isna()] = '알수없음'
/tmp/ipykernel_15432/905040304.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['가로'][i] = extract_height(data['사이즈'][i])
/tmp/ipykernel_15432/905040304.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['세로'][i] = extract_height(data['사이즈'][i])


### (7) 판매일자 계절 추출

In [36]:
data['판매계절'] = ''

def extract_season(date):
    month = int(date.split('-')[1])
    if month in [12, 1, 2]:
        return '겨울'
    elif month in [3,4,5]:
        return '봄'
    elif month in [6,7,8]:
        return '여름'
    elif month in [9,10,11]:
        return '가을'

data['판매계절'] = data['판매일자와판매처'].map(extract_season)

### (8) 판매처 추출

In [45]:
data['판매처'] = ''

def extract_space(space_):
    if ' ' in space_:
        return space_.split(' ')[-1]
    else:
        return space_

        
data['판매처'] = data['판매일자와판매처'].map(extract_space)
data['판매처'][data['판매처'] == '2020-09-21케이옥션'] = '케이옥션'

/tmp/ipykernel_15432/2958256712.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['판매처'][data['판매처'] == '2020-09-21케이옥션'] = '케이옥션'


### (9) 범주형 변수 Onehot encoding

In [46]:
dummies = pd.get_dummies(data.loc[:, ['작가생존여부','판매계절', '재료', '판매처']], drop_first=False)

### (9) 연속형 변수 정규화 (Min-max)

In [47]:
from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()
numeric_data =data.loc[:, ['가로', '세로', '작품 판매 횟수']]
numeric_data.loc[:, ['가로', '세로', '작품 판매 횟수']] = min_max_scaler.fit_transform(numeric_data)
numeric_data.head()

,가로,세로,작품 판매 횟수
0,0.065372,0.065372,0.0
1,0.026429,0.026429,0.0
2,0.018878,0.018878,0.0
3,0.025351,0.025351,0.0
4,0.019957,0.019957,0.0


### (10) X, y 정의

In [48]:
X = pd.concat([dummies, numeric_data], axis = 1)
y = data['판매가격']

In [49]:
X['판매가격'] = data['판매가격']
X['작가명'] = data['작가명']
X['제목'] = data['제목']
X['이미지 고유 번호'] = data['이미지 고유 번호']
X.to_excel('./data/final_data.xlsx', index =False, encoding = 'euc-kr')

del X['판매가격']
del X['작가명']
del X['제목']
del X['이미지 고유 번호']


### (11) Train Test 분할

In [6]:
from sklearn.model_selection import train_test_split
final_data = pd.read_excel('final_data_rgb_hsv.xlsx')

table_variable = ['작가생존여부_사망', '작가생존여부_생존', '작가생존여부_알수없음', '판매계절_가을', '판매계절_겨울', '판매계절_봄',
       '판매계절_여름', '재료_견본채색', '재료_기타', '재료_브론즈', '재료_비단에수묵담채', '재료_석판화',
       '재료_실크스크린', '재료_알수없음', '재료_오프셋석판화', '재료_종에이수묵담채', '재료_종이에먹', '재료_종이에수묵',
       '재료_종이에수묵담채', '재료_종이에수묵채색', '재료_종이에수채', '재료_종이에유채', '재료_지본묵서',
       '재료_지본수묵', '재료_지본채색', '재료_캔버스에아크릴', '재료_캔버스에유채', '재료_캔버스에혼합재료',
       '판매처_꼬모옥션', '판매처_마이아트옥션', '판매처_서울옥션', '판매처_아이옥션', '판매처_에이옥션', '판매처_칸옥션',
       '판매처_케이옥션', '판매처_헤럴드아트데이', '가로', '세로', '작품 판매 횟수']

X = final_data.loc[:, table_variable]
y = np.log10(final_data['판매가격'])


In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1004)

## 4. Modeling

In [32]:
from xgboost import XGBRegressor

xgb_model = XGBRegressor(n_estimators = 500, max_depth = 3, learning_rate = 0.01).fit(X_train, y_train)

#### (1) Train 성능

In [33]:
train_predict = xgb_model.predict(X_train)

In [34]:
from sklearn.metrics import mean_squared_error, r2_score

print("RMSE: {}".format(np.sqrt(mean_squared_error(y_train, train_predict))))
print("R2 Score: {}".format(r2_score(y_train, train_predict)))

RMSE: 0.5784673242548991
R2 Score: 0.28188206790606196


#### (2) Test 성능

In [35]:
test_predict = xgb_model.predict(X_test)

In [36]:
from sklearn.metrics import mean_squared_error, r2_score

print("RMSE: {}".format(np.sqrt(mean_squared_error(y_test, test_predict))))
print("R2 Score: {}".format(r2_score(y_test, test_predict)))

RMSE: 0.5775962381065534
R2 Score: 0.29688430957701306
